## Solution: Maximizing flow in a network

We generalize the problem in the following way: Given a directed graph $G=(V,A)$ with to special distinct vertices $s,t\in V$ (think of $s$ as the source and $t$ as the target) and arc capacities $u\colon A\to \mathbb{R}_{\geq 0}$, we want to find a flow value $f_a\geqslant 0$ for each $a\in A$ such that the following two conditions hold:

- **(Capacity constraints)** For no arc $a\in A$, the flow $f_a$ is larger than its capacity $u_a$.
- **(Conservation constraints)** For every vertex $v\in V\setminus\{s,t\}$, the total amount of flow on incoming arcs equals the total amount of flow on outgoing arcs.

Such flows are called $s$-$t$ flows. Moreover, among all flows satisfying these conditions, we want to find one maximizing the net outflow at $s$, i.e., the value of all flows on outgoing arcs at $s$ minus the value of all flows on incoming arcs at $t$.

---

It is easy to see that the above generalizes the given setting: Choosing the countries as vertices, i.e.,

$$V=\{F,G,C,S,A\},$$

and the roads as arcs, i.e.,

$$A=\{(S,F),(G,A),(C,A),(F,G),(F,C),(G,C)\},$$

the graph $G=(V,A)$ describes the given road network. Setting $s=S$ and $t=A$ corresponds to our wish to send flow from Spain to Austria.

---

### Coming up with an LP

To write a linear program that finds an $s$-$t$ flow in $G$ of maximum value, we introduce variables $f_a\in\mathbb{R}_{\geq0}$ for all $a\in A$. It is then easy to formulate capacity and conservation constraints as linear constraints:

- Capacity constraints: $$f_a\leq u_a \quad \text{for all } a\in A.$$
- Conservation constraints: $$\sum_{a\in \delta^-(v)} f_a = \sum_{a\in \delta^+(v)} f_a \quad\text{for all } v\in V\setminus\{s,t\}.$$

Here $\delta^-(v)$ and $\delta^+(v)$ denote all incoming and outgoing arcs at vertex $v$, respectively. Using this notation, we can also write the value of a flow $f$ as a linear function: It is precisely the net outflow at $s$, which is $$\sum_{a\in \delta^+(s)} f_a - \sum_{a\in \delta^-(s)} f_a.$$

Consequently, a linear program that solves the maximum $s$-$t$ flow problem is the following:

$$
\begin{array}{rrcll}
\max & \sum_{a\in \delta^+(s)} f_a - \sum_{a\in \delta^-(s)} f_a \\
     & f_a & \leq & u_a & \forall a\in A \\
     & \sum_{a\in \delta^+(v)} f_a - \sum_{a\in \delta^-(v)} f_a & = & 0 & \forall v\in V\setminus\{s,t\} \\
     & f_a & \geq & 0 & \forall a\in A.
\end{array}
$$

---

### Implementing and solving the LP

To solve the given concrete problem, we implement the above LP. Recall that the graph we were dealing with was defined as follows:

In [ ]:
import matplotlib.pyplot as plt
import networkx as nx
%matplotlib inline

# Create the graph
G = nx.DiGraph()
G.add_nodes_from(["F","G","C","S","A"])
vertex_pos = {"F": (0, .5),"G": (1, 1),"C": (1, 0),"S": (-1, 0.5),"A": (2, .5)}
G.add_edges_from([("S","F"),("G","A"),("C","A"),("F","G"),("F","C"),("G","C")])

# Display the capacities
nx.draw(G, vertex_pos, with_labels=True, font_size=15, arrowsize=20, node_color='y')

# Add edge capacities
capacities = dict({("S","F"): 5.25,("C","A"): 2.25,("G","A") :3.5,("F","G"): 3.75,("F","C"): 2.25,("G","C"): .5})
nx.draw_networkx_edge_labels(G, pos=vertex_pos, label_pos=0.5, edge_labels=capacities, font_size=15)

plt.show()

What we do below is written to work for any directed graph $G$ stored in a variable `G` and edge capacities stored in a dictionary `capacities`. Additionally, we need `s` and `t` to be set properly:

In [ ]:
s = 'S'
t = 'A'

We start with creating an empty maximization problem and all our variables.

In [ ]:
# import pulp
import pulp

# define an empty maximization problem
flowLP = pulp.LpProblem("Maximum flow problem", pulp.LpMaximize)

# create a dictionary of variables
f = dict([ [a, pulp.LpVariable(f"Flow on {a}", lowBound = 0)] for a in G.edges ])

Next, we construct the objective function:

In [ ]:
objective = pulp.lpSum([ f[a] for a in G.out_edges(s) ] 
                       + [ -f[a] for a in G.in_edges(s) ])
flowLP.setObjective(objective)

Looping over all arcs, we can add the capacity constraints:

In [ ]:
for a in G.edges:
    flowLP.addConstraint(f[a] <= capacities[a], f"capacity constraint on {a}")

The last family of constraints is the family of flow conservation constraints. Note that we do not need these at $s$ and $t$!

In [ ]:
for v in G.nodes:
    if v != s and v != t:
        flowLP.addConstraint(pulp.lpSum([ f[a] for a in G.out_edges(v) ] 
                                        + [ -f[a] for a in G.in_edges(v) ]) == 0,
                             f"conservation constraint at {v}")

Note that non-negativity constraints for the variables were already set when we defined the variables. To check that we have all constraints, let's have a look at them:

In [ ]:
flowLP.constraints

Finally, let's solve the LP and check the values:

In [ ]:
flowLP.solve()

optimum = flowLP.objective.value()

print(f"The value of the maximum flow from '{s}' to '{t}' is {optimum}.\n")

print("To achieve this, the company can transport goods as follows:")
for a in G.edges:
    print(f"Send a flow of {f[a].value():4.2f} units from '{a[0]}' to '{a[1]}'.")

Of course, we can also draw the corresponding flow.

In [ ]:
nx.draw(G, vertex_pos, with_labels=True, font_size=15, arrowsize=20, node_color='r')
flow = dict([ [a, f[a].value()] for a in G.edges ])
nx.draw_networkx_edge_labels(G, pos=vertex_pos, label_pos=0.5, edge_labels=flow, font_size=15)

plt.show()

---

### Bonus: Checking correctness using the built-in max flow algorithm

To check correctness of the maximum flow value, we can use the flow algorithm that comes with `networkx`. To this end, we turn the capacities into an edge attribute, and then call the algorithm with respect to these attributes.

In [ ]:
nx.set_edge_attributes(G, capacities, 'capacity')

In [ ]:
total_flow, edge_flow = nx.maximum_flow(G, s, t)

total_flow

The variable `total_flow` shold have the same value as the value that we computed above using the LP. Also here, we can visualize the flow:

In [ ]:
nx.draw(G, vertex_pos, with_labels=True, font_size=15, arrowsize=20, node_color='r')
flow = dict([ [a, edge_flow[a[0]][a[1]]] for a in G.edges ])
nx.draw_networkx_edge_labels(G, pos=vertex_pos, label_pos=0.5, edge_labels=flow, font_size=15)

plt.show()

You might observe that the actual flow is different from what we computed using the LP earlier! This is not a problem: The maximum flow value is uniquely determined, but the flow itself might not be unique.